In [12]:
import pandas as pd   
import requests
import json
import pyarrow
from datetime import datetime
import os 
import numpy as np

In [13]:
API_KEY = "7c503e5a0fb49718fbf6591b5536713d"

def cargar_json():
    with open("Traducc_Desc.json", "r", encoding="utf-8") as f:
        clima_data = json.load(f)
    return clima_data["Traducciones"], clima_data["Descripciones"]

Traducciones, Descripciones = cargar_json()


def get_clima (Ciudad, units:str = 'metric' ):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={Ciudad}&appid={API_KEY}&units={units}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error: {response.status_code} - {response.text}")
    
def transformar_datos(json_data):
    clima_en = json_data["weather"][0]["main"]
    descripcion_en = json_data["weather"][0]["description"]

    clima_es = Traducciones.get(clima_en, clima_en) 
    descripcion_es = Descripciones.get(descripcion_en.lower(), descripcion_en).capitalize()  

    datos = {
        "Ciudad": json_data.get("name"),
        "Fecha y Hora": datetime.fromtimestamp(json_data["dt"]),
        "Temperatura": json_data["main"]["temp"],
        "Sensación Térmica": json_data["main"]["feels_like"],
        "Humedad": json_data["main"]["humidity"],
        "Presión": json_data["main"]["pressure"],
        "Clima": clima_es,
        "Descripción": descripcion_es
    }

    df = pd.DataFrame([datos])
    return df



In [14]:
ciudades = ['Buenos Aires', 'Cordoba', 'Santa Fe', 'Mendoza', 'Salta', 'Entre Rios', 'San Luis', 'San Juan', 'Neuquen', 'La Rioja', 'Misiones']
data = []

for ciudad in ciudades:
    try:
            json_data = get_clima(ciudad)
            df = transformar_datos(json_data)
            data.append(df)
    except Exception as e:
            print(f"Error al obtener datos de {ciudad}: {e}")
        
    df_total = pd.concat(data, ignore_index=True)
df_total

,Ciudad,Fecha y Hora,Temperatura,Sensación Térmica,Humedad,Presión,Clima,Descripción
0,Buenos Aires,2025-04-21 15:55:20,19.76,19.31,58,1016,Despejado,Cielo despejado
1,Córdoba,2025-04-21 15:55:03,23.38,23.03,48,1011,Nublado,Pocas nubes
2,Santa Fe,2025-04-21 15:59:40,22.88,22.45,47,1013,Despejado,Cielo despejado
3,Mendoza,2025-04-21 15:53:13,21.74,21.09,43,1010,Despejado,Cielo despejado
4,Salta,2025-04-21 15:52:56,17.33,17.13,77,1012,Nublado,Nublado
5,Entre Rios,2025-04-21 16:01:50,30.48,31.96,51,1007,Nublado,Nubes dispersas
6,San Luis,2025-04-21 16:00:09,22.62,21.93,38,1010,Nublado,Nublado
7,San Juan,2025-04-21 15:56:54,24.97,24.28,29,1007,Despejado,Cielo despejado
8,Neuquén,2025-04-21 16:00:06,17.94,17.57,68,1011,Nublado,Nublado
9,La Rioja,2025-04-21 15:59:34,10.88,10.22,84,1016,Nublado,Nublado


In [15]:
df = df_total.copy()

temp_prom = df.groupby('Ciudad')['Temperatura'].mean().round(2)

sens_prom = df.groupby('Ciudad')['Sensación Térmica'].mean().round(2)

humedad_prom = df.groupby('Ciudad')['Humedad'].mean().round(2)

df['Diferencia termica'] = abs(df['Temperatura'] - df['Sensación Térmica'])
df_ter_prom = df.groupby('Ciudad')['Diferencia termica'].mean().round(2)

resumen = pd.DataFrame({
    'Temperatura promedio (°C)': temp_prom,
    'Sensacion termica promedio (°C)': sens_prom,
    'Humedad promedio (%)': humedad_prom,
    'Diferencia termica promedio (°C)': df_ter_prom
})
resumen

,Temperatura promedio (°C),Sensacion termica promedio (°C),Humedad promedio (%),Diferencia termica promedio (°C)
Ciudad,,,,
Buenos Aires,19.76,19.31,58.0,0.45
Córdoba,23.38,23.03,48.0,0.35
Entre Rios,30.48,31.96,51.0,1.48
La Rioja,10.88,10.22,84.0,0.66
Mendoza,21.74,21.09,43.0,0.65
Misiones,17.99,17.49,63.0,0.50
Neuquén,17.94,17.57,68.0,0.37
Salta,17.33,17.13,77.0,0.20
San Juan,24.97,24.28,29.0,0.69


In [16]:
df = pd.read_parquet('datos/raw/clima_2025-04-18_13-23.parquet')
df

,Ciudad,Fecha y Hora,Temperatura,Sensación Térmica,Humedad,Presión,Clima,Descripción
0,Buenos Aires,2025-04-18 13:21:15,19.37,19.16,69,1017,Nublado,Nubes rotas
1,Córdoba,2025-04-18 13:19:57,16.82,16.86,88,1018,Nublado,Nublado
2,Santa Fe,2025-04-18 13:21:48,22.78,22.94,70,1013,Nublado,Pocas nubes
3,Mendoza,2025-04-18 13:16:50,18.74,17.69,39,1020,Nublado,Nubes dispersas
4,Salta,2025-04-18 13:15:52,25.88,25.78,48,1015,Nublado,Pocas nubes
5,Entre Rios,2025-04-18 13:21:51,33.44,34.26,39,1011,Nublado,Pocas nubes
6,San Luis,2025-04-18 13:21:15,15.50,15.25,82,1018,Nublado,Nublado
7,San Juan,2025-04-18 13:21:52,18.84,18.37,61,1019,Nublado,Nublado
8,Neuquén,2025-04-18 13:21:17,17.94,16.68,34,1021,Despejado,Cielo despejado
9,La Rioja,2025-04-18 13:21:54,15.88,15.09,60,1005,Nublado,Pocas nubes


In [18]:
def analizar_con_numpy(df):
    temperaturas = df["Temperatura"].to_numpy()
    media = np.mean(temperaturas)
    desvio = np.std(temperaturas)
    print(f"Media de temperatura: {media:.2f}°C")
    print(f"Desvío estándar: {desvio:.2f}")
analizar_con_numpy(df)

Media de temperatura: 20.08°C
Desvío estándar: 5.19
